#### Decision tree for allocating oil type to ATB traffic
see google drawing (ATB_Oil_Attribution)[https://docs.google.com/drawings/d/1Aigd2wccnl6-pQVo4KLN0xq1is6poD9z5JnFVb-fduE/edit ] for a visual representation

In [1]:
# These will become function inputs
origin = 'Westridge Marine Terminal'
destination = 'Pacific'
ship_type = 'atb'
random_seed=None

In [2]:
import numpy 
import yaml
import pathlib

In [3]:
# load the list of CAD, US and generic origins and destinations
master_dir  = '/Users/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/monte_carlo/'
master_file = 'master.yaml'

with open(f'{master_dir}{master_file}') as file:
    master = yaml.safe_load(file)

# Assign US and CAD origin/destinations from master file
CAD_origin_destination = master['categories']['CAD_origin_destination']
US_origin_destination = master['categories']['US_origin_destination']

# Get file paths to fuel-type yaml files
# US_origin is for US as origin
# US_combined represents the combined import and export of fuel
home = pathlib.Path(master['directories'])
CAD_yaml     = home/master['files']['CAD_origin']
WA_in_yaml   = home/master['files']['WA_destination']
WA_out_yaml  = home/master['files']['WA_origin']
US_yaml      = home/master['files']['US_origin']
USall_yaml   = home/master['files']['US_combined']
Pacific_yaml = home/master['files']['Pacific_origin']

#### Create decision-making tree for tanker traffic
All attributions will be tank-cargo at this point.  I don't think we have decision-making around spill type (fuel or cargo) yet (need to check!).  Will likely add here

In [4]:
def get_oil_type_cargo(yaml_file, facility):
     with yaml_file.open("rt") as file:
            cargo = yaml.safe_load(file)
            ship = cargo[facility][ship_type]
            probability = [ship[fuel]['fraction_of_total'] for fuel in ship]    
            fuel_type = random_generator.choice(list(ship.keys()), p = probability)
            return fuel_type

In [5]:
# these pairs need to be used together for "get_oil_type_cargo" (but don't yet have error-checks in place):
# - "WA_in_yaml" and "destination"
# - "WA_out_yaml" and "origin"

# Need to add a catch for erroneous cases where origin-destination in AIS analysis
# pairs vessel-type and facility to null values in the DOE transfer data. 
# For these cases (which shouldn't happen but might), we will use the generic US fuel allocations

# Initialize PCG-64 random number generator
random_generator = numpy.random.default_rng(random_seed)

if origin in CAD_origin_destination:
    if origin == 'Westridge Marine Terminal':
        if destination == 'U.S. Oil & Refining':
            fuel_type = get_oil_type_cargo(CAD_yaml, origin)
        elif destination in US_origin_destination:
            fuel_type = get_oil_type_cargo(CAD_yaml, origin)
        elif destination in CAD_origin_destination:
            # assume export within CAD is from Jet fuel storage tanks 
            # as there is a pipeline to Parkland for crude oil
            fuel_type = 'jet'
        else:
            fuel_type = get_oil_type_cargo(CAD_yaml, origin)
    else:
        if destination in US_origin_destination:
            # we have better information on WA fuel transfers, so I'm prioritizing this information source
            fuel_type = get_oil_type_cargo(WA_in_yaml, destination)
        elif destination == 'ESSO Nanaimo Departure Bay':
            fuel_type = get_oil_type_cargo(CAD_yaml, destination)
        elif destination == 'Suncor Nanaimo':
            fuel_type = get_oil_type_cargo(CAD_yaml, destination)
        else: 
            fuel_type = get_oil_type_cargo(CAD_yaml, origin)
elif origin in US_origin_destination:
    if destination == 'Westridge Marine Terminal':
        # Westridge stores jet fuel from US for re-distribution 
        fuel_type = 'jet'
    else:
        fuel_type = get_oil_type_cargo(WA_out_yaml, origin)
elif destination in US_origin_destination:
    fuel_type = get_oil_type_cargo(WA_in_yaml, destination)
elif destination in CAD_origin_destination:
    if destination == 'Westridge Marine Terminal':
        # Westridge doesn't receive crude for storage 
        fuel_type = 'jet'
    else:
        fuel_type = get_oil_type_cargo(CAD_yaml, destination)
elif origin == 'Pacific':
    fuel_type = get_oil_type_cargo(Pacific_yaml, origin)
elif origin == 'US':
    fuel_type = get_oil_type_cargo(US_yaml, origin)
else:
    # For all other traffic, use a generic fuel attribution from the combined
    # US import and export
    fuel_type = get_oil_type_cargo(USall_yaml, origin)